# Abstract Interpretation

To prove program correctness using *axiomatic semantics*, i.e., Hoare logic, we need to provide *loop invariants*, which as we have seen, can be a difficult and error-prone process.
Thus, axiomatic semantics, while extremely powerful and used in many verification tools, e.g., Microsoft Dafny, is not an automatic, push-button approach.
In addition, axiomatic semantic approach relies on checking validity of verification conditions.
For complex programs, this checking process can be very expensive and in many cases not feasible.

In this lecture, we will look at *abstract interpretation*, a verification approach that attempts to address the limiations of requiring loop invariants and expensive formula checkings.


## Domain Example:  {Zero, Odd, Even} over Natural numbers


- Maps from concrete natural numbers to either {0, odd, even} 
- Examples:  5 -> odd  , 4 ->  even ,  0 -> 0

In [4]:
def mult(A,B):
    # P = {A >= 0 and B >= 0}
    x = A
    y = B
    z = 0
    # L1
    while x > 0:
        if x % 2 == 1:
            z = z + y 
        x = x // 2
        y = y * 2;
        # L2

    # L3
    return z


- Consider the `mult` example above that multiplies two natural numbers $A, B$.  Our goal is to find, at the end of the program, at location `L3`, the values of $x, y, z$ using the abstract domains **{Zero, Odd, Even}** over natural numbers.


- Case:  `A = odd, B = odd`

|      |loc | x | y | z |
|:---  |:--- |:---:|:---:|:---:|
| Init                   | L1 | odd     | odd  | odd  |
|Iter 1||||
| - entrance |L2 | odd     | odd  | odd  |
| - body: case 1, x is odd  | L3 | $\top$  | even | odd  |
| - body: case 2, x is even (not possible)|L3| $\bot$| $\bot$ | $\bot$|
| - body: union of cases | L3 | $\top$ | even | odd|
| if loop exits| L4| 0 | even | odd |
|Iter 2||||
| - entrance |L2 | $\top$     | even  | odd  |
| - body: case 1, x is odd  | L3 | $\top$  | even | odd  |
| - body: case 2: x is even | L3| odd/even | even | odd |
| - union of cases | L3 | $\top$ | even | odd|
| if loop loop exits| L4| 0 | even | odd |
|Iter 3: entrance same as iter 2: fixed point||||
| - entrance |L2 | $\top$     | even  | odd  |
  Final results |||
| union of loop exits | L4 | 0 | even | odd


    

Abstract transformers

| $>$|    |    |    | 
|:--:|:--:|:--:|:---:
|    | 0  | odd| even
|0   | f  | f  | f   
|odd | t  | t/f| t/f 
|even| t  | t/f| t/f 


| $+$|    |    |    |
|:--:|:--:|:--:|:---:
|    | 0  | odd | even
|0   | 0  | odd | even
|odd | 0  | even| odd 
|even| 0  | odd | even



## Domain Example: Interval over Integers
- Concrete Domain:  Z,   e.g.,  x = 1
- Abstract Domain: [Z, Z], e.g.,  x = [1,1]

### Functions
- Abstract function $\alpha$: convert a set of concrete values to an abstract value
    -  E.g., $\alpha(10) = [10, 10], \alpha(\{-1,3, 0\}) = [-1, 3]$


- Concreterize function $\gamma$:  convert an abstract value to a set of concrete values
  - E.g.,  $\gamma([1,1]) = 1, \gamma([-1,3]) = \$-1,0,1,2,3\}$
  

- Transfer functions
    - Arithmetic operations
        - $+: [a1, a2] + [b1, b2]= [a1+b1, a2+b2]$
    - Union: 
        - $\cup: [a1, a2] \cup [b1, b2] = [min(a1,b1), max(a2,b2)]$
    - Intersection:
        - $\cap: [a1, a2] \cap [b1, b2] = [max(a1,b2), max(a2,b2)]$
    

- Another example

```C
x = 5;
// x = [5,5]
y = x - 1
// x = [5,5], y = [4,4]
```

```C
//x = [-10, 10]
y = x - 1
// y = ??
```

- Example

```C

//L1
if (x <= 4) then
    //L2:  
    x = -10
    //L3    
else
    //L4
    x = 100
    //L5
//L6
```



|     | x   |  Reason             |
|:---:|:---:|:-------------------|
|L1   |[0,7]| assume              |   
|L2   |[0,4]| [0,7] $\cap$ [-$\inf$, 4] |
|L3   |[-10,10   ]| [0,4] $\cup$ [-10, 10]                    |
|L4   | [5,7]    |  [0,7] $\cap$ [5,$\inf$]                    |
|L5   |  |   |             |
|L6   |     ||

Resources
- http://www2.informatik.uni-freiburg.de/~heizmann/ProgramVerification/slides/20111025-Tue.pdf
